In [19]:
import pandas as pd

from bokeh.io import show, output_notebook
from bokeh.layouts import widgetbox, row
from bokeh.models.widgets import RadioGroup
from bokeh.models import CustomJS ,ColumnDataSource
from bokeh.plotting import figure, output_file

In [20]:
df = pd.read_excel('TPbars.xlsx')
baseline=df.iloc[0:1,0:4]
s1=df.iloc[1:2,0:4] # water saving
s2=df.iloc[2:3,0:4] # lw
s3=df.iloc[3:4,0:4] # water saving and lw

s1["Scenario"] = "Improved rose"
s2["Scenario"] = "Improved rose"
s3["Scenario"] = "Improved rose"


df

,Scenario,Retail price,Social cost,Water cost
0,Normal rose,10,3,5
1,Water saving rose,9,3,2
2,Living wage rose,11,1,5
3,Water saving & living wage rose,10,1,2


In [21]:
#social cost of scenario 1
values[1]["Retail price"]


1    9
Name: Retail price, dtype: int64

In [22]:
#can only be executed once because it alters column names
values = (baseline,s1,s2,s3)
#change social cost to social cost top (social cost + retail cost)
for i in values:
   i["Social cost"]=i["Social cost"]+i["Retail price"]
   i.rename(index=str, columns={"Social cost": "Social cost top"}, inplace = True)
#change env cost to env cost top (env cost + social cost top)
for i in values:
   i["Water cost"]=i["Water cost"]+i["Social cost top"]
   i.rename(index=str, columns={"Water cost": "Water cost top"}, inplace = True)


In [23]:
plotframe=ColumnDataSource(df)
plotframe

ColumnDataSource(id='eed3d96d-ca22-4eff-87d0-eb582563eea9', ...)

In [24]:
normal = ColumnDataSource(baseline)
normal1 = ColumnDataSource(s1)
improved1 = ColumnDataSource(s1)
improved2 = ColumnDataSource(s2)
improved3 = ColumnDataSource(s3)




scenarios

retail=list(df["Retail price"])
retail

In [41]:
plot = figure(plot_width=600, plot_height=400, x_range=["Normal rose","Improved rose"], y_range=(0, 20), title="Price of a Kenyan rose", tools="", logo="grey")

bluenormalbar= plot.vbar(
    x="Scenario", 
    width=0.5, 
    top="Retail price", 
    color="MidnightBlue", 
    legend="Market price",
    visible=True,
    source=normal
    )
blueimprovedbar= plot.vbar(
    x="Scenario", 
    width=0.5, 
    top="Retail price",
    color="MidnightBlue", 
    legend="Market price",
    visible=True,
    source=normal1
    )

rednormalbar= plot.vbar(
    x="Scenario", 
    width=0.5, 
    top="Social cost top",
    bottom="Retail price",
    color = "FireBrick", 
    legend="Social cost",
    source=normal
    )

redimprovedbar= plot.vbar(
    x="Scenario", 
    width=0.5, 
    top="Social cost top",
    bottom="Retail price",
    color = "FireBrick", 
    legend="Social cost",
    source=normal1
)
greennormalbar= plot.vbar(
    x="Scenario", 
    width=0.5, 
    top="Water cost top",
    bottom="Social cost top",
    color="SeaGreen", 
    legend="Market price",
    visible=True,
    source=normal
    )
greenimprovedbar= plot.vbar(
    x="Scenario", 
    width=0.5, 
    top="Water cost top",
    bottom="Social cost top",
    color="SeaGreen", 
    legend="Market price",
    visible=True,
    source=normal1
    )


In [42]:
#this should change the source data of blueimproved, back and forth, 
callback = CustomJS(args=(dict(improved=blueimprovedbar, s1=improved1, s2=improved2, s3=improved3)), code="""
    var source = improved.data_source;

    if (cb_obj.active == 0) {
        source.data = s1.data
    } else if (cb_obj.active == 1) {
        source.data = s2.data
    } else if (cb_obj.active == 2) {
        source.data = s3.data
    }
    improved.trigger('change');
    """)

In [43]:
widgets = RadioGroup(
    labels=["Water saving", "Living wage","Water saving + Living wage"], 
    callback=callback,
    active=0
)

In [44]:
output_notebook()
#output_file("widgets.html")
improved1.data

Loading BokehJS ...

{'Retail price': [9],
 'Scenario': ['Improved rose'],
 'Social cost top': [12],
 'Water cost top': [14],
 'index': ['1']}

In [45]:
layout=(row(widgetbox(widgets),plot))
show(layout)

In [11]:
improved1.data

{'Retail price': [9],
 'Scenario': ['Improved rose'],
 'Social cost': [3],
 'Water cost': [2],
 'index': [1]}

In [12]:
improved2.data

{'Retail price': [11],
 'Scenario': ['Improved rose'],
 'Social cost': [1],
 'Water cost': [5],
 'index': [2]}

In [13]:
'''
plot.vbar(
    x=[scenarios], 
    width=0.5, 
    top=df["Retail price"]+df["Social cost"], 
    color = "FireBrick", 
    bottom = df["Retail price"], 
    legend="Social cost")
plot.vbar(
    x=[scenarios], 
    width=0.5, 
    top=df["Retail price"]+df["Social cost"]+df["Water cost"], 
    color = "SeaGreen", 
    bottom = df["Retail price"]+df["Social cost"], 
    legend="Water cost"
    '''

'\nplot.vbar(\n    x=[scenarios], \n    width=0.5, \n    top=df["Retail price"]+df["Social cost"], \n    color = "FireBrick", \n    bottom = df["Retail price"], \n    legend="Social cost")\nplot.vbar(\n    x=[scenarios], \n    width=0.5, \n    top=df["Retail price"]+df["Social cost"]+df["Water cost"], \n    color = "SeaGreen", \n    bottom = df["Retail price"]+df["Social cost"], \n    legend="Water cost"\n    '